# First-Class Functions

## Table of Content

### Introduction

First-class objects

* Created at runtime
* Assigned to a variable or element in a data structure
* Passed as an argument to a function
* Returned as the result of a function

Treating a Functino like an object

In [5]:
# Example 5.1
# here we return the function, therefore treating it as a first-class object

def factorial(n):
    "Returns n!"
    return 1 if n < 2 else n * factorial(n - 1)

In [6]:
factorial.__doc__

'Returns n!'

In [7]:
fact = factorial # assigning function to a variable

In [8]:
fact

<function __main__.factorial(n)>

In [9]:
fact(5)

120

In [10]:
map(factorial, range(11)) # lazy evaluation?

In [11]:
list(map(fact, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

## Higher-order functions

Functions that take a function as argument or return a function as the result

map is an example, factorial as well

## Modern replacement for map, filter and reduce

map and filter has been somewhat replaced by list comprehension and generator expressions. Listcomp and genexp does the job of map and filter combined but is more readable.

In [12]:
from functools import reduce
from operator import add

reduce(add, range(100))

4950

## Anonymous functions

To use a higher order function, it often makes sense to use a small, one-off function, this is frequently the use case for lambda functions.

Lambda functions are often intentionally unwiedly for anything more than a simple function. Thus it is not useful outside the input to a higher order function.

To refactor any piece of code that is hard to understand because of a lambda, follow Lundh's lambda refactoring recipe

1. Write a comment explaining what the lambda does
2. Study the comment for clues on what the lambda function is doing and think of a name that captures the essence of the comment
3. Convert the lambda to a def statement with the name you picked
4. Remove the comment

## The seven flavors of callable objects

A call operator () and you can check whether an object is callable by using callable()

1. User-defined functions
def or lambda expressions
2. Built-in functions
function implement in C of CPython, e.g len or time.strftime
3. Built-in methods
methods implemented in C, like dict.get
4. Methods
Functions defined in the body of a class
5. Classes
When invoked, a class runs its `__new__` method to create an instance, then `__init__` to initialize it, and finally the instance is returned to the caller
6. Class instances
If a class defines a `__call__` method, then its instances may be invoked as functions
7. Generator functions
Functions or methods that use the yield keyword. When called, return a generator object

In [37]:
## Turning in instance into a callable simply requires __call__
# You'd want to use this when some internal state must be kept across invocations, like the remaining items in a BingoPick

class BingoPick:
    def __init__(self, items):
        self._items = items
        from random import shuffle
        shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoPick')
        
    def __call__(self):
        return self.pick()
    
# Decorators are often used to keep the internal state as well
# They are functions but it is sometimes convenient to remember between calls of the decorator
# e.g memoization (caching the results of expensive function calls)

def memoize(fn):
    cache = {}
    def inner(*args):
        if args not in cache:
            cache[args] = fn(*args)
        return cache[args]
    return inner

@memoize
def fib(n):
    return n if n < 2 else fib(n - 1) + fib(n - 2)

huh = fib(10)

In [38]:
# view the cache
fib.__closure__[0].cell_contents

{(1,): 1,
 (0,): 0,
 (2,): 1,
 (3,): 2,
 (4,): 3,
 (5,): 5,
 (6,): 8,
 (7,): 13,
 (8,): 21,
 (9,): 34,
 (10,): 55}

In [ ]:
fib = memoize(fib)
memoize(fib)
cache = {}
cache[args] = fn(*args) # stores the results in cache[args] = fn(*args)
# computes and returns the result because inner returns cache[args]
# Takes in a function

In [40]:
def f(*args): # does * args here create tuple?
    print(args) # creates tuple of arguments

f(1, 2, 3)

(1, 2, 3)


In [44]:
def memoize(fn):
    cache = {}
    def inner(*args):
        if args not in cache:
            cache[args] = fn(*args)
        return cache[args]
    return inner

@memoize
def fib(n):
    return n if n < 2 else fib(n - 1) + fib(n - 2)

huh = fib(10, 5, 3)

TypeError: fib() takes 1 positional argument but 3 were given

In [45]:
def test(huh):
    return huh
test.random = 5

In [47]:
test.random

5

In [50]:
def example(*args):
    print(*args)
    print(args)

*args: Captures arguments as a tuple in args

**kwargs: Captures keyword arguments as a dictionary in kwargs

Entries into kwargs must be valid identifiers. Which means they must start with a letter or underscore and contain only letters, digits, and underscores.

In [77]:
def add(a, b):
    return a + b

In [79]:
# method 1

values = {'a': 1, 'b': 2}
s = add(**values) # this unpacks it for named arguments
print(s)

3


In [81]:
# method 2 double unpacking
# * unpacks a sequence or collection into positional arguments
values_1 = (1, 2)
values_2 = {'c': 1, 'd': 2}

def add(a, b, c, d):
    return a + b + c + d

s = add(*values_1, **values_2)
print(s)

6


## Using it in the function signature is a different matter

e.g f(*args, **kwargs)

In [83]:
def add(*values):
    s = 0
    print(values)
    for v in values:
        s += s + v
    return s

values = [1, 2, 3, 4]
s = add(*values)
print(s)

(1, 2, 3, 4)
26


In [88]:
# Usage 4

def add(*values, **values_d):
    s = 0
    print(values)
    print(values_d)
    for v in values:
        s += v
    print(s)
    for v in values_d.values():
        print(f'v is {v}, s is {s}')
        s += v
    return s

values = [1, 2, 3, 4]
vd = {'a': 1, 'b': 2, 'c': 3}
s = add(*values, **vd)
print(s)

(1, 2, 3, 4)
{'a': 1, 'b': 2, 'c': 3}
10
v is 1, s is 10
v is 2, s is 11
v is 3, s is 13
16
